In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import os
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
%matplotlib inline 
import cv2 as cv
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential 
from keras.layers import Conv2D,MaxPool2D,DepthwiseConv2D,Flatten,Dense,Dropout,AveragePooling2D,BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.utils import to_categorical
from tensorflow.keras import regularizers

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
img_height=512
img_width=512
batch_size=128
directory="../input//dl-hack-track-1-cv/train/"

In [3]:
datagen=ImageDataGenerator(
    rescale=1.0/255,
#     featurewise_center=True,
#     featurewise_std_normalization=True,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
    horizontal_flip=False,
    data_format="channels_last",
    validation_split=0.2,
    dtype=tf.float32
    
    
)

In [4]:
train_data=datagen.flow_from_directory(
    "../input//dl-hack-track-1-cv/train/",
    target_size=(512,512),
    batch_size=batch_size,
    color_mode="rgb",
    class_mode='categorical',
    shuffle=True,
    subset='training'
    
)
val_data=datagen.flow_from_directory(
    "../input//dl-hack-track-1-cv/train/",
    target_size=(512,512),
    batch_size=batch_size,
    color_mode="rgb",
    class_mode='categorical',
    shuffle=True,
    subset='validation'
    
)


Found 28000 images belonging to 2 classes.
Found 7000 images belonging to 2 classes.


In [5]:
# model= keras.Sequential()
# # input shape is shape of images
# model.add(keras.Input(shape=(512,512,3)))
# model.add(DepthwiseConv2D(3,(2,2)))
# model.add(BatchNormalization())
# model.add(Conv2D(32,(3,3),activation = 'relu',padding='same')) 
# model.add(BatchNormalization())
# model.add(MaxPool2D((2, 2)))
# model.add(Conv2D(48, (3, 3), activation='relu', padding='same'))
# model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
# model.add(MaxPool2D((2, 2)))
# model.add(Conv2D(80, (3, 3), activation='relu', padding='same'))
# model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
# model.add(MaxPool2D((2, 2)))
# model.add(Flatten())
# model.add(Dense(64, activation='relu'))
# model.add(Dense(16,activation="relu"))
# model.add(Dense(2,activation="sigmoid"))
# model.compile(
#         optimizer=tf.keras.optimizers.Adam(0.00005),
#             loss='sparse_categorical_crossentropy',
#             metrics=['accuracy']
#     )
# model.summary()
# #===========================================================

# batches=1
# q=True
# for x_batch, y_batch in train_data:
#         if q:
#             model.fit(x_batch, y_batch)
#             q=False
#         else:
#             model.evaluate(x_batch,y_batch)
#             q=True
#         batches += 1
        
#         if batches >= 70:
#             # we need to break the loop by hand because
#             # the generator loops indefinitely
#             break

In [6]:
def inflow(i):
    #x=Conv2D(3,(3,3),padding="same")(i)
    x=DepthwiseConv2D((2,2),strides=(2,2))(i)
    x=BatchNormalization()(x)
    #x=Conv2D(3,(3,3),padding="same",activation="relu")(x)
    return x
def outflow(x):
    x=Flatten()(x)
    x=Dense(64,activation='relu')(x)
    x=Dense(16,activation='relu')(x)
    x=Dense(2,activation="sigmoid")(x)
    return x

vgg19 =keras.applications.VGG16(weights='imagenet',input_shape=(256,256,3), include_top=False) 
# for layer in vgg19.layers:
#     layer.trainable=False

    
def midflow(x):
    for layer in vgg19.layers:
        #layer.trainable = False
        x=layer(x)
    return x
inputs=keras.Input(shape=(512,512,3))
y=inflow(inputs)
y=midflow(y)
outputs=outflow(y)
Model= keras.Model(inputs,outputs)
Model.summary()

58892288/58889256 [==============================] - 2s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 512, 512, 3)]     0         
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 256, 256, 3)       15        
_________________________________________________________________
batch_normalization (BatchNo (None, 256, 256, 3)       12        
_________________________________________________________________
input_1 (InputLayer)         multiple                  0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
______________________________________________________________

In [8]:
Model.compile(
       optimizer=tf.keras.optimizers.Adam(0.000003),
       loss='categorical_crossentropy',
       metrics=['accuracy']
)
#Model.load_weights("transfer.h5")
Model.fit(train_data,epochs=1,validation_data=val_data)
Model.save("transfer.h5")

219/219 [==============================] - 1081s 5s/step - loss: 0.0160 - accuracy: 0.9956 - val_loss: 0.0289 - val_accuracy: 0.9889


In [11]:
"""
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_2 (InputLayer)         [(None, 512, 512, 3)]     0         
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 256, 256, 3)       15        
_________________________________________________________________
batch_normalization (BatchNo (None, 256, 256, 3)       12        
_________________________________________________________________
Vgg-16
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                2097216   
_________________________________________________________________
dense_1 (Dense)              (None, 16)                1040      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 34        
=================================================================
Total params: 16,813,005
Trainable params: 16,812,999
Non-trainable params: 6



Model.compile(
       optimizer=tf.keras.optimizers.Adam(0.000003),
       loss='categorical_crossentropy',
       metrics=['accuracy']
)
#Model.load_weights("transfer.h5")
Model.fit(train_data,epochs=1,validation_data=val_data)
Model.save("transfer.h5")



"""

'\nModel: "model"\n_________________________________________________________________\nLayer (type)                 Output Shape              Param #   \n=================================================================\ninput_2 (InputLayer)         [(None, 512, 512, 3)]     0         \n_________________________________________________________________\ndepthwise_conv2d (DepthwiseC (None, 256, 256, 3)       15        \n_________________________________________________________________\nbatch_normalization (BatchNo (None, 256, 256, 3)       12        \n_________________________________________________________________\nVgg-16\n_________________________________________________________________\nflatten (Flatten)            (None, 32768)             0         \n_________________________________________________________________\ndense (Dense)                (None, 64)                2097216   \n_________________________________________________________________\ndense_1 (Dense)              (None,

In [16]:
def classify(Id):
    x=predictwithId(Id)
    if x>0.975:
        return 0.975
    if x<0.025:
        return 0.025
    return float(x)
    

def predictwithId(Id):
    path='../input/dl-hack-track-1-cv/test/'+str(Id)+".png"
    image = tf.keras.preprocessing.image.load_img(path,
                                              grayscale=False, 
                                              color_mode="rgb", 
                                              target_size=None,
                                              interpolation="nearest")
    input_arr = keras.preprocessing.image.img_to_array(image)
    input_arr = np.array(input_arr)/255  # Convert single image to a batch.
    result=Model.predict(np.reshape(input_arr,(1,512,512,3)))[:,1]
    return result


In [15]:
path='../input/dl-hack-track-1-cv/train/fake/'+str(4)+".png"
image = tf.keras.preprocessing.image.load_img(path,
                                              grayscale=False, 
                                              color_mode="rgb", 
                                              target_size=None,
                                              interpolation="nearest")
input_arr = keras.preprocessing.image.img_to_array(image)
input_arr = np.array(input_arr)/255  # Convert single image to a batch.
result=Model.predict(np.reshape(input_arr,(1,512,512,3)))[:,1]
float(result)

3.058799620703212e-06

In [ ]:
submission=pd.read_csv("../input/dl-hack-track-1-cv/sample_submission.csv")
submission['p_real']=submission['id'].apply(classify)
print(len(submission))
submission.set_index('id').to_csv('sample_submission.csv')

In [ ]:
Model.save("transfer.h5")

In [ ]:
path='../input/dl-hack-track-1-cv/test/'+str(0)+".png"
image = tf.keras.preprocessing.image.load_img(path,
                                              grayscale=False, 
                                              color_mode="rgb", 
                                              target_size=None,
                                              interpolation="nearest")
input_arr = keras.preprocessing.image.img_to_array(image)
input_arr = np.array(input_arr)/255  # Convert single image to a batch.
input_arr = np.reshape(input_arr,(1,512,512,3))/255
model2 = tf.keras.models.load_model("m2.h5")
blocks = [7]
outputs = [model2.layers[i].output for i in blocks]
model2 = keras.Model( inputs= model2.inputs, outputs = outputs)
feature_map = model2.predict(input_arr)

for i,fmap in zip(blocks,feature_map):
    fig = plt.figure(figsize=(20,15))
    #https://stackoverflow.com/a/12444777
    fig.suptitle("BLOCK_{}".format(i) , fontsize=20)
    for i in range(1,features.shape[3]+1):
        plt.subplot(8,8,i)
        plt.imshow(fmap[:,:,i-1] , cmap='gray')
        #print(fmap[:,:,i].shape)
plt.show()

In [ ]:

def inflow(i):
    x=Conv2D(3,3,padding="same")(i)
    x=DepthwiseConv2D((2,2),strides=(2,2))(x)
    return x
vgg_model=keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(256,256,3))
model_aug=Sequential()
model_aug.add(Conv2D(3,3,padding="same",input_shape=(512,512,3)))
model_aug.add(DepthwiseConv2D((2,2),strides=(2,2)))
model_aug.add(vgg_model)
top_model=Sequential()


top_model.add(Flatten(input_shape=(2, 2, 512)))
#model_aug.add(Dropout(0.3))
top_model.add(Dense(64, activation='relu'))

top_model.add(Dense(1, activation='sigmoid'))

model_aug.add(top_model)

for layer in model_aug.layers[3].layers[:17]:
    layer.trainable=False
model      